### **Búsqueda y Minería de Información 2022-23**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Implementación de un motor de búsqueda**

Fechas:

* Comienzo: martes 7 / jueves 9 de febrero
* Entrega: martes 21 / jueves 23 de febrero (14:00)

# Introducción

## Autores

Xu Chen

Ana Martínez Sabiote

## Objetivos

Los objetivos de esta práctica son:

* La iniciación a la implementación de un motor de búsqueda.
*	Una primera comprensión de los elementos básicos necesarios para implementar un motor completo.
*	La iniciación al uso de la librería [Whoosh](https://whoosh.readthedocs.io/en/latest/intro.html) en Python para la creación y utilización de índices, funcionalidades de búsqueda en texto.
*	La iniciación a la implementación de una función de ránking sencilla.

Los documentos que se indexarán en esta práctica, y sobre los que se realizarán consultas de búsqueda serán documentos HTML, que deberán ser tratados para extraer y procesar el texto contenido en ellos. 

La práctica plantea como punto de partida una pequeña API general sencilla (y cuyo uso se puede ver en un programa de prueba que se encuentra al final del enunciado), que pueda implementarse de diferentes maneras, como así se hará en esta práctica y las siguientes. A modo de toma de contacto y arranque de la asignatura, en esta primera práctica se completará una implementación de la API utilizando Whoosh, con lo que resultará bastante trivial la solución (en cuanto a la cantidad de código a escribir). En la siguiente práctica el estudiante desarrollará sus propias implementaciones, sustituyendo el uso de Whoosh que vamos a hacer en esta primera práctica.

En términos de operaciones propias de un motor de búsqueda, en esta práctica el estudiante se encargará fundamentalmente de:

a) En el proceso de indexación: recorrer los documentos de texto de una colección dada, eliminar del contenido posibles marcas tales como html, y enviar el texto a indexar por parte de Whoosh. 

b) En el proceso de responder consultas: implementar una primera versión sencilla de una o dos funciones de ránking en el modelo vectorial, junto con alguna pequeña estructura auxiliar.

## Material proporcionado

Se proporcionan (bien en el curso de Moodle o dentro de este documento):

*	Varias clases e interfaces Python (mayormente incompletas) a lo largo de este *notebook*, desde las que el estudiante partirá para completar código e integrará con ellas las suyas propias. 
La celda de prueba *al final de este notebook* implementa un programa que deberá funcionar con el código a implementar por el estudiante. Además, se proporciona a continuación una celda con código ejemplo que ilustra las funciones más útiles de la API de Whoosh.
*	Una pequeña colección <ins>docs1k.zip</ins> con aproximadamente 1.000 documentos HTML, y un pequeño fichero <ins>urls.txt</ins>. Ambas representan colecciones de prueba para depurar las implementaciones y comprobar su corrección.
*	Un documento de texto <ins>output.txt</ins> con la salida estándar que deberá producir la ejecución de la celda de prueba (salvo los tiempos de ejecución que pueden cambiar, aunque la tendencia en cuanto a qué métodos tardan más o menos debería cumplirse).

## Ejemplo API Whoosh

En la siguiente celda de código se incluyen varios ejemplos para comprobar cómo usar la API de la librería *Whoosh*.

In [40]:
# Whoosh API
import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os, os.path
import shutil

Document = Schema(
        path=ID(stored=True),
        content=TEXT(vector=Format))

def whooshexample_buildindex(dir, urls):
    if os.path.exists(dir): shutil.rmtree(dir)
    os.makedirs(dir)
    writer = whoosh.index.create_in(dir, Document).writer()
    for url in urls:
        writer.add_document(path=url, content=BeautifulSoup(urlopen(url).read(), "lxml").text)
    writer.commit()

def whooshexample_search(dir, query):
    index = whoosh.index.open_dir(dir)
    searcher = index.searcher()
    qparser = QueryParser("content", schema=index.schema)
    print("Search results for '", query, "'")
    for docid, score in searcher.search(qparser.parse(query)).items():
        print(score, "\t", index.reader().stored_fields(docid)['path'])
    print()

def whooshexample_examine(dir, term, docid, n):
    reader = whoosh.index.open_dir(dir).reader()
    print("Total nr. of documents in the collection:", reader.doc_count())
    print("Total frequency of '", term, "':", reader.frequency("content", term))
    print("Nr. documents containing '", term, "':", reader.doc_frequency("content", term))
    for p in reader.postings("content", term).items_as("frequency") if reader.doc_frequency("content", term) > 0 else []:
        print("\tFrequency of '", term, "' in document", p[0], ":", p[1])
    raw_vec = reader.vector(docid, "content")
    raw_vec.skip_to(term)
    if raw_vec.id() == term:
        print("Frequency of '", raw_vec.id(), "' in document", docid, reader.stored_fields(docid)['path'], ":", raw_vec.value_as("frequency"))
    else:
        print("Term '", term, "' not found in document", docid)
    print("Top", n, "most frequent terms in document", docid, reader.stored_fields(docid)['path']) 
    vec = reader.vector(docid, "content").items_as("frequency")
    for p in sorted(vec, key=lambda x: x[1], reverse=True)[0:n]:
        print("\t", p)
    print()

urls = ["https://en.wikipedia.org/wiki/Simpson's_paradox", 
        "https://en.wikipedia.org/wiki/Bias",
        "https://en.wikipedia.org/wiki/Entropy"]

dir = "index/whoosh/example/urls"

whooshexample_buildindex(dir, urls)
whooshexample_search(dir, "probability")
whooshexample_examine(dir, "probability", 0, 5)

Search results for ' probability '
1.4655068943024385 	 https://en.wikipedia.org/wiki/Simpson's_paradox
1.4195698196433444 	 https://en.wikipedia.org/wiki/Entropy
0.6570661342415882 	 https://en.wikipedia.org/wiki/Bias

Total nr. of documents in the collection: 3
Total frequency of ' probability ': 26.0
Nr. documents containing ' probability ': 3
	Frequency of ' probability ' in document 0 : 9
	Frequency of ' probability ' in document 1 : 1
	Frequency of ' probability ' in document 2 : 16
Frequency of ' probability ' in document 0 https://en.wikipedia.org/wiki/Simpson's_paradox : 9
Top 5 most frequent terms in document 0 https://en.wikipedia.org/wiki/Simpson's_paradox
	 ('paradox', 53)
	 ('simpson', 51)
	 ('data', 25)
	 ('two', 19)
	 ('displaystyle', 17)



## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado.  

El peso de la nota de esta práctica en la calificación final de prácticas es del **20%**.

La calificación se basará en a) el **número** de ejercicios realizados y b) la **calidad** de los mismos. 
La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) **sin ninguna modificación**. El profesor comprobará este aspecto ejecutando la celda de prueba así como otras pruebas adicionales.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**. Si se necesita entregar algún fichero adicional (por ejemplo, imágenes) se puede subir un fichero ZIP a la tarea correspondiente de Moodle. En cualquiera de los dos casos, el nombre del fichero a subir será **bmi-p1-XX**, donde XX debe sustituirse por el número de pareja (01, 02, ..., 10, ...).

En concreto, se debe documentar:

- Qué version(es) del modelo vectorial se ha(n) implementado en el ejercicio 2.
- Cómo se ha conseguido colocar un documento en la primera posición de ránking, para cada buscador implementado en el ejercicio 2.
- El trabajo realizado en el ejercicio 3. 
- Y cualquier otro aspecto que el estudiante considere oportuno destacar.


## Indicaciones

Se podrán definir clases adicionales a las que se indican en el enunciado, por ejemplo, para reutilizar código. Y el estudiante podrá utilizar o no el software que se le proporciona, con la siguiente limitación: 

*	No deberá editarse el código proporcionado más allá de donde se indica explícitamente.
*	**La celda de prueba deberá ejecutar** correctamente sin ninguna modificación.

# Ejercicio 1: Implementación basada en Whoosh

Implementar las clases y módulos necesarios para que la celda de prueba funcione. Se deja al estudiante deducir alguna de las relaciones jerárquicas entre las clases Python.

## Ejercicio 1.1: Indexación (3.5pt)

Definir las siguientes clases:

* Index: clase general (no depende de Whoosh) y que encapsule los métodos necesarios para que funcione la celda de prueba que se encuentra al final del enunciado.
* Builder: clase general (no depende de Whoosh) que permite construir un índice (a través del método Builder.build()), tal y como se llama desde la celda de prueba entregada.
* WhooshIndex: clase que cumpla con la interfaz definida en *Index* usando la librería de whoosh.
* WhooshBuilder: clase que cumpla con la interfaz definida en *Builder* pero que use internamente la librería de whoosh.

La entrada para construir el índice (método Builder.build()) podrá ser, tal y como se puede ver en el programa de prueba al final de este notebook, a) un fichero de texto con direcciones Web (una por línea); b) una carpeta del disco (se indexarán todos los ficheros de la carpeta, sin entrar en subcarpetas); o c) un archivo zip que contiene archivos comprimidos a indexar. Para simplificar, supondremos que el contenido a indexar es siempre HTML.

In [41]:
class Index:
    def __init__(self, index_path):
        self.index_path = index_path

    def doc_freq(self, term):
        pass

    def all_terms_with_freq(self):
        pass

    def ndocs(self):
        pass

    def all_terms(self):
        pass

    def total_freq(self, term):
        pass

    def doc_freq(self, doc_id):
        pass

    def term_freq(self, term, doc_id):
        pass


class Builder:
    def __init__(self, index_path):
        self.index_path = index_path
        pass

    # Collection es un string
    def build(self, collection):
        pass

    def commit(self):
        pass

In [42]:
import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser

import zipfile

# A schema in Whoosh is the set of possible fields in a document in the search space. 
# We just define a simple 'Document' schema, with a path (a URL or local pathname)
# and a content.
Document = Schema(
        path=ID(stored=True),
        content=TEXT(vector=Format))

class WhooshBuilder(Builder):
    def build(self, collection):
        if os.path.exists(self.index_path): shutil.rmtree(self.index_path)
        os.makedirs(self.index_path)
        self.writer = whoosh.index.create_in(self.index_path, Document).writer()

        if os.path.isdir(collection):
            for file in os.listdir(collection):
                filepath = os.path.join(collection, file)

                with open(filepath, "r") as f:
                    self.writer.add_document(path=filepath, content=BeautifulSoup(f.read(), "lxml").text)

        elif zipfile.is_zipfile(collection):
            with zipfile.ZipFile(collection, 'r') as zp:
                for name in zp.namelist():
                    self.writer.add_document(path=name, content=BeautifulSoup(zp.read(name), "lxml").text)
        elif collection.endswith(".txt"):
            with open(collection, "r") as f:
                for line in f.readlines():
                    self.writer.add_document(path=line, content=BeautifulSoup(urlopen(line).read(), "lxml").text)

    def commit(self):
        self.writer.commit()

class WhooshIndex(Index):
    def __init__(self, index_path):
        self.index_path = index_path
        self.reader = whoosh.index.open_dir(self.index_path).reader()

    # Numero de documentos en los que aparece term
    def doc_freq(self, term):
        return self.reader.doc_frequency("content", term)

    def all_terms_with_freq(self):
        result = []
        for term in self.all_terms():
            result.append((term, self.total_freq(term)))
        return result

    def ndocs(self):
        return self.reader.doc_count()

    def all_terms(self):
        return list(self.reader.field_terms("content"))

    def total_freq(self, term):
        return self.reader.frequency("content", term)

    def term_freq(self, term, doc_id):
        raw_vec = self.reader.vector(doc_id, "content")
        raw_vec.skip_to(term)
        if raw_vec.id() == term:
            return raw_vec.value_as("frequency")
        else:
            return 0

    def doc_path(self, doc_id):
        return self.reader.stored_fields(doc_id)['path']

### Explicación/documentación

*(por hacer)*

## Ejercicio 1.2: Búsqueda (2pt)

Implementar la clase WhooshSearcher como subclase de Searcher.

In [43]:
import math
from abc import ABC, abstractmethod
import re

def from_query_to_terms(text):
    return re.findall(r"[^\W\d_]+|\d+", text.lower())

"""
    This is an abstract class for the search engines
"""
class Searcher(ABC):
    def __init__(self, index):
        self.index = index
    @abstractmethod
    def search(self, query, cutoff):
        """ Returns a list of documents built as a pair of path and score.
            As a simplification, the query can be divided in terms by considering blank spaces. 
            Moreover, the terms can be normalized to lower case (i.e., you may use function 'from_query_to_terms').
        """

In [44]:
class WhooshSearcher(Searcher):
    def __init__(self, index_path):
        self.index_path = index_path
        self.WhooshIndex = WhooshIndex(index_path)
        self.index = whoosh.index.open_dir(self.index_path)
        
    def search(self, query, cutoff):
        searcher = self.index.searcher()
        qparser = QueryParser("content", schema=self.index.schema)

        results = [(self.WhooshIndex.doc_path(docid), score) for docid, score in searcher.search(qparser.parse(query)).items()]
        results.sort(key=lambda tup: tup[1], reverse=True)

        return results[0:cutoff]

### Explicación/documentación

*(por hacer)*

# Ejercicio 2: Modelo vectorial

Implementar dos modelos de ránking propios, basados en el modelo vectorial.

## Ejercicio 2.1: Producto escalar (2.5pt)

Implementar un modelo vectorial propio que utilice el producto escalar (sin dividir por las normas de los vectores) como función de ránking, por medio de la clase VSMDotProductSearcher, como subclase de Searcher.

Este modelo hará uso de la clase Index y se podrá probar con la implementación WhooshIndex (puedes ver un ejemplo de esto en la celda de prueba).

Además, la clase VSMDotProductSearcher será intercambiable con WhooshSearcher, como se puede ver en la celda de prueba, donde la función test_search utiliza una implementación u otra sin distinción.

In [45]:
class VSMDotProductSearcher(Searcher):
    ## TODO ##
    # Your code here #

SyntaxError: unexpected EOF while parsing (71648419.py, line 3)

### Explicación/documentación

*(por hacer)*

### Ejercicio

Añadir a mano un documento a la colección docs1k.zip de manera que aparezca el primero para la consulta “obama family tree” para este buscador. Documentar cómo se ha conseguido y por qué resulta así.

*(por hacer)*

## Ejercicio 2.2: Coseno (2pt)

Refinar la implementación del modelo para que calcule el coseno, definiendo para ello una clase VSMCosineSearcher. Para ello se necesitará extender Builder (o WhooshBuilder) con el cálculo de los módulos de los vectores, que deberán almacenarse en un fichero, en la carpeta de índice junto a los ficheros que genera cada índice. 

Pensad en qué parte del diseño interesa hacer esto, en concreto, qué clase y en qué momento tendría que calcular, devolver y/o almacenar estos módulos.

In [ ]:
class VSMCosineSearcher(VSMDotProductSearcher):
    ## TODO ##
    # Your code here #

### Explicación/documentación

*(por hacer)*

### Ejercicio

Añadir a mano un documento a la colección docs1k.zip de manera que aparezca el primero para la consulta “obama family tree” para este buscador. Documentar cómo se ha conseguido y por qué resulta así.

*(por hacer)*

# Ejercicio 3: Estadísticas de frecuencias (1pt)

Utilizando las funcionalidades de la clase Index, implementar una función term_stats que calcule a) las frecuencias totales en la colección de los términos, ordenadas de mayor a menor, y b) el número de documentos que contiene cada término, igualmente de mayor a menor. Visualizar las estadísticas obtenidas en dos gráficas en escala log-log (dos gráficas por cada colección, seis gráficas en total), que se mostrarán en el cuaderno entregado.

De esta forma, podrás comprobar si las estadísticas de la colección siguen algún tipo de comportamiento esperado (como la conocida [Ley de Zipf](https://es.wikipedia.org/wiki/Ley_de_Zipf)).

In [ ]:
def term_stats(index):
    ## TODO ##
    # Your code here #

### Explicación/documentación

*(por hacer)*

# Celda de prueba

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta *collections* en el mismo directorio que este *notebook*. El fichero *toy.zip* hay que descomprimirlo para indexar la carpeta que contiene.

In [47]:
import os
import shutil
import time

def clear (index_path: str):
    if os.path.exists(index_path): shutil.rmtree(index_path)
    else: print("Creating " + index_path)
    os.makedirs(index_path)

def test_collection(collection_paths: list, index_path: str, word: str, query: str):
    start_time = time.time()
    print("=================================================================")
    print("Testing indices and search on " + str(len(collection_paths)) + " collections")

    # Let's create the folder if it did not exist
    # and delete the index if it did
    clear(index_path)

    # We now test building an index
    test_build(WhooshBuilder(index_path), collection_paths)

    # We now inspect the index
    index = WhooshIndex(index_path)
    test_read(index, word)

    print("------------------------------")
    print("Checking search results")
    test_search(WhooshSearcher(index_path), query, 5)
    #test_search(VSMDotProductSearcher(WhooshIndex(index_path)), query, 5)
    #test_search(VSMCosineSearcher(WhooshIndex(index_path)), query, 5)

def test_build(builder, collections: list):
    stamp = time.time()
    print("Building index with", type(builder))
    for collection in collections:
        print("Collection:", collection)
        # This function should index the received collection and add it to the index
        builder.build(collection)
    # When we commit, the information in the index becomes persistent
    # we can also save any extra information we may need
    # (and that cannot be computed until the entire collection is scanned/indexed)
    builder.commit()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def test_read(index, word):
    stamp = time.time()
    print("Reading index with", type(index))
    print("Collection size:", index.ndocs())
    print("Vocabulary size:", len(index.all_terms()))
    terms = index.all_terms_with_freq()
    terms.sort(key=lambda tup: tup[1], reverse=True)
    print("  Top 5 most frequent terms:")
    for term in terms[0:5]:
        print("\t" + term[0] + "\t" + str(term[1]) + "=" + str(index.total_freq(term)))
    print()
    # More tests
    doc_id = 0
    print()
    print("  Frequency of word \"" + word + "\" in document " + str(doc_id) + " - " + index.doc_path(doc_id) + ": " + str(index.term_freq(word, doc_id)))
    print("  Total frequency of word \"" + word + "\" in the collection: " + str(index.total_freq(word)) + " occurrences over " + str(index.doc_freq(word)) + " documents")
    print("  Docs containing the word'" + word + "':", index.doc_freq(word))
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_search (engine, query, cutoff):
    stamp = time.time()
    print("  " + engine.__class__.__name__ + " for query '" + query + "'")
    for path, score in engine.search(query, cutoff):
        print(score, "\t", path)
    print()
    print("Done (", time.time() - stamp, "seconds )")
    print()


index_root_dir = "./index/"
collections_root_dir = "./collections/"
test_collection ([collections_root_dir + "toy/"], index_root_dir + "toy", "cc", "aa dd")
test_collection ([collections_root_dir + "urls.txt"], index_root_dir + "urls", "wikipedia", "information probability")
test_collection ([collections_root_dir + "docs1k.zip"], index_root_dir + "docs", "seat", "obama family tree")
test_collection ([collections_root_dir + "toy/", collections_root_dir + "urls.txt", collections_root_dir + "docs1k.zip"], index_root_dir + "all_together", "seat", "obama family tree")

Testing indices and search on 1 collections
Creating ./index/toy
Building index with <class '__main__.WhooshBuilder'>
Collection: ./collections/toy/
Done ( 0.017806291580200195 seconds )

Reading index with <class '__main__.WhooshIndex'>
Collection size: 4
Vocabulary size: 39
  Top 5 most frequent terms:
	aa	9.0=9.0
	bb	5.0=5.0
	sleep	5.0=5.0
	cc	3.0=3.0
	die	2.0=2.0


  Frequency of word "cc" in document 0 - ./collections/toy/d1.txt: 2
  Total frequency of word "cc" in the collection: 3.0 occurrences over 2 documents
  Docs containing the word'cc': 2
Done ( 0.0017817020416259766 seconds )

------------------------------
Checking search results
  WhooshSearcher for query 'aa dd'

Done ( 0.0024726390838623047 seconds )

Testing indices and search on 1 collections
Creating ./index/urls
Building index with <class '__main__.WhooshBuilder'>
Collection: ./collections/urls.txt
Done ( 2.710953712463379 seconds )

Reading index with <class '__main__.WhooshIndex'>
Collection size: 3
Vocabulary s

### Salida obtenida por el estudiante

*(por hacer)*